## Analyzing the impact of weather on bike trip duration using OLS regression method



In [42]:
import pandas as pd

pd.set_option('display.max_columns', 999)

# All Libraries needed for Regression imported

# All Dataframes as CSV needed for regression

bike_journey_data=pd.read_csv('bike_journey_data_27_mar_2020.csv')
weather_data=pd.read_csv('weather_data_27_mar_2020.csv')


In [43]:
# Collpase the bike and weather dataset by trips by hour and sum the number of trips and total duration of all trips by hour

reg_merge= bike_journey_data.merge(right=weather_data,
                             left_on = 'id_hours',
                             right_on = 'id_Hours')

reg_merge['bike_trips_by_hour'] = 1
def bike_trips_by_hour(reg_merge, renamed_count):
    bike_trips_by_hour = reg_merge.loc[:, ('id_hours', 'bike_trips_by_hour')].groupby(['id_hours']).sum()
    bike_trips_by_hour.reset_index(inplace=True)
    bike_trips_by_hour.rename(columns={'bike_trips_by_hour': str(renamed_count)}, inplace=True)
    return bike_trips_by_hour
bike_trips_by_hour = bike_trips_by_hour(reg_merge, 'bike_trips_by_hour')

reg_merge3= reg_merge.merge(right=bike_trips_by_hour,
                             left_on = 'id_hours',
                             right_on = 'id_hours')

reg_merge3['Duration in hours'] = 1
def duration_by_hour(reg_merge3, renamed_count):
    duration_by_hour = reg_merge3.loc[:, ('id_hours', 'Duration in minutes')].groupby(['id_hours']).sum()
    duration_by_hour.reset_index(inplace=True)
    duration_by_hour.rename(columns={'Duration in minutes': str(renamed_count)}, inplace=True)
    return duration_by_hour
duration_by_hour = duration_by_hour(reg_merge3, 'duration_by_hour')

regression_dataset= reg_merge3.merge(right=duration_by_hour,
                             left_on = 'id_hours',
                             right_on = 'id_hours')

regression_dataset['Avg trip duration']=regression_dataset['duration_by_hour']/regression_dataset['bike_trips_by_hour_y']


In [45]:
df_month=pd.get_dummies(regression_dataset.Month)
df_month.columns=['Jan','Feb','Mar', 'Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
#df_month=df_month.astype(int)
regression_dataset = regression_dataset.join(df_month)

In [46]:
regression_dataset["temp_0"] = regression_dataset['temperature'].apply(lambda x: 1 if x <=0 else 0)
regression_dataset["temp_5"] = regression_dataset['temperature'].apply(lambda x: 1 if x>0 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=5 else 0)
regression_dataset["temp_10"] = regression_dataset['temperature'].apply(lambda x: 1 if x>5 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=10 else 0)
regression_dataset["temp_15"] = regression_dataset['temperature'].apply(lambda x: 1 if x>10 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=15 else 0)

regression_dataset["temp_20"] = regression_dataset['temperature'].apply(lambda x: 1 if x>15 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=20 else 0)
regression_dataset["temp_25"] = regression_dataset['temperature'].apply(lambda x: 1 if x>20 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=25 else 0)

regression_dataset["temp_30"] = regression_dataset['temperature'].apply(lambda x: 1 if x>25 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=30 else 0)
regression_dataset["temp_35"] = regression_dataset['temperature'].apply(lambda x: 1 if x>30 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=35 else 0)
regression_dataset["temp_40"] = regression_dataset['temperature'].apply(lambda x: 1 if x>35 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=40 else 0)

regression_dataset.head()

,Unnamed: 0,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,Start Date Converted,End Date Converted,Hours_x,Week Day,Day,Month,Year,Duration in minutes,id_x,id_hours,Date,temperature,precipIntensity,humidity,windSpeed,visibility,Date Converted,Hours_y,id_y,id_Hours,bike_trips_by_hour_x,bike_trips_by_hour_y,Duration in hours,duration_by_hour,Avg trip duration,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,Oct,Nov,Dec,temp_0,temp_5,temp_10,temp_15,temp_20,temp_25,temp_30,temp_35,temp_40
0,0,83284852,660,8282,02/01/2019 17:47,94,"Bricklayers Arms, Borough",02/01/2019 17:36,197,"Stamford Street, South Bank",2019-01-02,2019-01-02,17,2,2,1,2019,11.0,2019-1-2,2019-01-02-17,2019-01-02 17:00:00,4.0,0.0,0.67,5.17,16.093,2019-01-02,17,2019-1-2,2019-01-02-17,1,2579,1,39342.0,15.25475,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,17,83284638,600,14513,02/01/2019 17:42,230,"Poured Lines, Bankside",02/01/2019 17:32,109,"Soho Square , Soho",2019-01-02,2019-01-02,17,2,2,1,2019,10.0,2019-1-2,2019-01-02-17,2019-01-02 17:00:00,4.0,0.0,0.67,5.17,16.093,2019-01-02,17,2019-1-2,2019-01-02-17,1,2579,1,39342.0,15.25475,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,35,83283348,840,15508,02/01/2019 17:16,334,"Concert Hall Approach 1, South Bank",02/01/2019 17:02,140,"Finsbury Square , Moorgate",2019-01-02,2019-01-02,17,2,2,1,2019,14.0,2019-1-2,2019-01-02-17,2019-01-02 17:00:00,4.0,0.0,0.67,5.17,16.093,2019-01-02,17,2019-1-2,2019-01-02-17,1,2579,1,39342.0,15.25475,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,122,83284637,480,419,02/01/2019 17:40,123,"St. John Street, Finsbury",02/01/2019 17:32,123,"St. John Street, Finsbury",2019-01-02,2019-01-02,17,2,2,1,2019,8.0,2019-1-2,2019-01-02-17,2019-01-02 17:00:00,4.0,0.0,0.67,5.17,16.093,2019-01-02,17,2019-1-2,2019-01-02-17,1,2579,1,39342.0,15.25475,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,145,83284256,780,9726,02/01/2019 17:35,807,"Bevington Road West, North Kensington",02/01/2019 17:22,176,"Gloucester Terrace, Bayswater",2019-01-02,2019-01-02,17,2,2,1,2019,13.0,2019-1-2,2019-01-02-17,2019-01-02 17:00:00,4.0,0.0,0.67,5.17,16.093,2019-01-02,17,2019-1-2,2019-01-02-17,1,2579,1,39342.0,15.25475,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [69]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

y = 'Avg trip duration'
x = ['temp_0','temp_5', 'temp_10', 'temp_15', 'temp_20', 'temp_25', 'temp_30', 'temp_35', 'Feb']
#model_Simple = sm.OLS(temperature_10_dataset[y], temperature_10_dataset[x]).fit()

model_1 = sm.OLS(regression_dataset[y], sm.add_constant(regression_dataset[x])).fit()

model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Avg trip duration   R-squared:                       0.192
Model:                            OLS   Adj. R-squared:                  0.192
Method:                 Least Squares   F-statistic:                 2.677e+05
Date:                Fri, 27 Mar 2020   Prob (F-statistic):               0.00
Time:                        21:56:55   Log-Likelihood:            -3.2345e+07
No. Observations:            10138355   AIC:                         6.469e+07
Df Residuals:                10138345   BIC:                         6.469e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         23.6070      0.045    524.667      0.000      23.519      23.695
temp_0        -8.5785      0.052   -164.877      0.000      -8.680      -8.476
temp_5        -7.7591      0.046   -170.123      0.000      -7.848      -7.670
temp_10       -6.8227      0.045   -150.996      0.000      -6.911      -6.734
temp_15       -5.4707      0.045   -121.172      0.000      -5.559      -5.382
temp_20       -3.5478      0.045    -78.590      0.000      -3.636      -3.459
temp_25        0.7241      0.045     16.016      0.000       0.636       0.813
temp_30        1.2383      0.046     26.672      0.000       1.147       1.329
temp_35        2.9489      0.048     61.128      0.000       2.854       3.043
Feb            0.8575      0.008    113.555      0.000       0.843       0.872
==============================================================================
Omnibus:                 14520728.212   Durbin-Watson:                   0.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      37724219259.869
Skew:                           7.777   Prob(JB):                         0.00
Kurtosis:                     301.430   Cond. No.                         81.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [66]:
dfoutput = summary_col([model_1],stars=True)
print(dfoutput)


          Avg trip duration
---------------------------
const     -16283645.6148   
          (293085993.2476) 
temp_0    -12.3452***      
          (0.0537)         
temp_5    -11.9279***      
          (0.0461)         
temp_10   -10.9932***      
          (0.0450)         
temp_15   -8.7667***       
          (0.0448)         
temp_20   -5.4208***       
          (0.0445)         
temp_25   -0.6567***       
          (0.0444)         
temp_30   0.4239***        
          (0.0455)         
temp_35   2.0217***        
          (0.0473)         
Jan       16283673.0569    
          (293085993.2447) 
Feb       16283673.8371    
          (293085993.2448) 
Mar       16283673.1546    
          (293085993.2463) 
Apr       16283674.7885    
          (293085993.2470) 
May       16283672.0886    
          (293085993.2445) 
Jun       16283671.4371    
          (293085993.2438) 
Jul       16283669.2219    
          (293085993.2473) 
Aug       16283670.6555    
          (29308599